# TensorFlow Data **Validation**

TensorFlow Data Validation (TFDV) is a library for exploring and validating  machine learning data. It is designed to be highly scalable and to work  well with TensorFlow and TensorFlow Extended (TFX). This notebook will explore the features of TFDV using the [Energy consumption of the Netherlands](https://www.kaggle.com/lucabasa/dutch-energy) dataset from Kaggle.

### Importing the data

Tensorflow works with datafrom Csv files or TFRecords, the dataset is composed by several Csv files, it contains contains information about eletricity and Gas. 

In the analysis it will be use the eletricity data for **Liander** and **Stedin** regions, the data is available in my Google Drive (Google colab has integration tools with Gdrive), but you can download at the Link above in Kaggle Datasets. In the analysis we'll use the Liander region as *Train* dataset and the Stedin region as *Test* dataset, the split between train and test dataset will be use to validate the schema define by TFDV.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
# Import libraries
from __future__ import print_function
import sys, os
import tempfile, urllib, zipfile

# Confirm it is using Python 2 due TFDV just works on python 2
assert sys.version_info.major is 2, 'Oops, not running Python 2'

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'Electricity')

# Download the zip file from GCP and unzip it
zip = '/content/gdrive/My Drive/Datasets/dutch-energy.zip'
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -lR {os.path.join(BASE_DIR, 'Electricity')}

Here's what we downloaded:
/tmp/tmp8yErFG/Electricity:
total 299208
-rw-r--r-- 1 root root  8710467 Jan 16 13:15 enexis_electricity_01012010.csv
-rw-r--r-- 1 root root  9434887 Jan 16 13:15 enexis_electricity_01012011.csv
-rw-r--r-- 1 root root  9357040 Jan 16 13:15 enexis_electricity_01012012.csv
-rw-r--r-- 1 root root  9645723 Jan 16 13:15 enexis_electricity_01012013.csv
-rw-r--r-- 1 root root  9680904 Jan 16 13:15 enexis_electricity_01012014.csv
-rw-r--r-- 1 root root  9721703 Jan 16 13:15 enexis_electricity_01012015.csv
-rw-r--r-- 1 root root  9657332 Jan 16 13:15 enexis_electricity_01012016.csv
-rw-r--r-- 1 root root 10110527 Jan 16 13:15 enexis_electricity_01012017.csv
-rw-r--r-- 1 root root 12399093 Jan 16 13:15 enexis_electricity_01012018.csv
-rw-r--r-- 1 root root 12824884 Jan 16 13:15 liander_electricity_01012009.csv
-rw-r--r-- 1 root root 12902559 Jan 16 13:15 liander_electricity_01012010.csv
-rw-r--r-- 1 root root 12932278 Jan 16 13:15 liander_electricity_01012011.csv
-rw-r

### TFDV

The TFDV is not available on the default packages in Google Colab, we install it  using pip

In [3]:
!pip install -q tensorflow_data_validation
import tensorflow_data_validation as tfdv

print('TFDV version: {}'.format(tfdv.version.__version__))

apache-beam 2.9.0 has requirement pytz<=2018.4,>=2018.3, but you'll have pytz 2018.9 which is incompatible.
TFDV version: 0.11.0


### Compute and visualize statistics

TFDV can compute descriptive statistics providing a quick overview of the data in terms of the features, shapes and distribution of the values.

First use tfdv.generate_statistics_from_csv to compute statistics for train data (Liander region), TFDV can compute descriptive statistics, generate_statistics_from_csv works specifying a file pattern. The pattern can contain glob characters  (*, ?, [...] and sets).

In [4]:
# Generate stats for Ste
train_stats = tfdv.generate_statistics_from_csv(data_location=DATA_DIR+'/l*')

# Visualize
tfdv.visualize_statistics(train_stats)

it is possible to check the distribution of each variable (max, min, median, mean, std and zeros), how much missing data to each feature. The numeric and categorical variables are shown separated.

The charts show the purchase_are has 64,81% missing data, cliking in the opção "Show Raw Data" it is possible to see the details results to each feature.

### Infer a schema

Infering a schema we save a lot of initial work, the scheme defines the data that are relevant to the model, for example, the type of each feature (numerical or categorical), for categorical features the schema also defines the domain - the list of acceptable values, TFDV provides a initial version of the schema based on the descriptive statistics.

Define the schema is important because the mode pipeline relies on it also it is provide a documentation of the data.

In [6]:
# Define the schame
schema = tfdv.infer_schema(statistics=train_stats)

# Display the schema 
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'zipcode_to',BYTES,required,,-
'city',BYTES,required,,-
'purchase_area',STRING,optional,single,'purchase_area'
'delivery_perc',FLOAT,required,,-
'smartmeter_perc',FLOAT,required,,-
'zipcode_from',BYTES,required,,-
'net_manager',STRING,required,,'net_manager'
'num_connections',INT,required,,-
'type_of_connection',STRING,required,,'type_of_connection'


,Values
Domain,
'purchase_area','Liander NW'
'net_manager',"'Liander N.V.', 'Liander N.V. (NW)', 'Liander N.V. (ZW)'"
'type_of_connection',"'1x10', '1x25', '1x35', '1x50', '1x6', '1x63', '3x25', '3x35', '3x50', '3x63', '3x80', 'OBK'"


### Check evaluation data for errors

We checked and infer a schema for the train dataset, the schema also should be consistent in the test dataset, it should have the same range of values in the numerical features, if the data is different between the test and train data we'll have problems in the model.

In [7]:
# Compute stats for evaluation data
test_stats = tfdv.generate_statistics_from_csv(data_location=DATA_DIR+'/s*')

# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=test_stats, 
                          rhs_statistics=train_stats,
                          lhs_name='Test_Dataset', 
                          rhs_name='Train_Dataset')

/usr/local/lib/python2.7/dist-packages/tensorflow_data_validation/statistics/generators/numeric_stats_generator.py:108: RuntimeWarning: overflow encountered in long_scalars
  numeric_stats.sum_of_squares += v * v


In evalation of the data TFDV shows in the plot both the train and test stats overlaid, it is possible to check the distribuition visually, it can speedy the change in the schema when necessary.

Looking in the results, we can infer easily the data from Liander region can't be use to predict the values on Stedin region, all the median, meadn, std, missing values are different between the dataset, but now we don't make any change and we will continue with the TFDV analysis.

### Anomalies

After generate stats for both train and test dataset, it necessary to check if the schema define to each one is the same (range of values, type of feature and so on), TFDV has a function to look for anomalies, e.g, features define different for each schema.

In [8]:
# Check eval data for errors by validating the test data stats using the 
# previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

# Display anomalies
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'net_manager',Expected data of type: BYTES but got INT,
'type_conn_perc',Expected data of type: INT but got STRING,
'purchase_area',Unexpected string values,"Examples contain values missing from the schema: Stedin (~52%), Stedin Delfland (~7%), Stedin Elektriciteit Zuid-Kennemerland (<1%), Stedin Midden-Holland (~4%), Stedin Schiedam (~1%), Stedin Utrecht (~30%), Stedin Weert (<1%)."


Three features has inconsistent values, two of them has wrong type (net_manager, type_conn_perc), the purchase_are it is a feature expected to be outside of the train schema, it is due the fact correspond to a different region in Netherlands.

### Fixing the erros

The corrections will depend of our knowlegde of the data, looking at each feature and decide what action should be take, looking in the description of each feature will to decide how to correct the erros, **net_manager** is code of the regional network manager, **type_conn_perc: percentage** of presence of the principal type of connection in the zipcode range, **purchase_area** code of the area where the energy is purchased.

For the type_conn_perc let's covert it to BYTES and append the missing values to the purcha_are.

In [9]:
# Convert the type_conn_perc to Bytes
type_conn_perc = tfdv.get_feature(schema, 'type_conn_perc')
type_conn_perc.type = 1 # Type 1 represent BYTES

# Add new value to the domain of feature net_manager.
purchase_area = tfdv.get_domain(schema, 'purchase_area')
purchase_area.value.append('Stedin')
purchase_area.value.append('Stedin Delfland')
purchase_area.value.append('Stedin Elektriciteit Zuid-Kennemerland')
purchase_area.value.append('Stedin Midden-Holland')
purchase_area.value.append('Stedin Schiedam')
purchase_area.value.append('Stedin Utrecht')
purchase_area.value.append('Stedin Weert')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(test_stats, schema)
tfdv.display_anomalies(updated_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'net_manager',Expected data of type: BYTES but got INT,


It is left just one error, but we don't need to convert the net_manager to Integer, we'll use this feature as Bytes, since net_manager correspond to a code

### Schema Environments

TFDV it is possible to define different schema according to your data, for example, you added labels in the train dataset but for prediction it is not necessary.

**Environments** can be used to express such requirements, features in schema can be associated with a set of environments using `default_environment`, `in_environment` and `not_in_environment`.

For our example we'll not use this function.

### Drift and Skew

TFDV also provides additional functionalities to detect drift and skew comparing the statistics of the different datasets, for Skew analysis TFDV has support to three kind (Schema Skew, Feature Skew and Distribuition Skew)

This feature are used in advanced analysis and are not focus of the notebook, for more information please visite [TFDV](https://www.tensorflow.org/tfx/data_validation/) website.

### Freeze Schema

After all ajusts is time to save for future use.

In [10]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(BASE_DIR)
schema_file = os.path.join(BASE_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "zipcode_to"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "city"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "purchase_area"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "purchase_area"
  presence {
    min_count: 1
  }
}
feature {
  name: "delivery_perc"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "smartmeter_perc"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "zipcode_from"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "net_manager"
  type: BYTES
  domain: "net_manager"
  presence {
    min_fraction: 1.0
    

### Conclusion

TFDV deliveres what it is promissed, it shows a summary of your data and helps to define the schema for using in TFX, when working with several files and data it take time to undestand all the data and make assumptions, after you spend more time defining what to do for each feature, after defining the model you can refine using TDFV you have all your features visually in front you it easy to undestand the changes. TFDV helps in this task let it more clean and inside a flow in the tensforflow ecosystem. It was a great addition by Google.

For completed information and new releases in TFDV please check the Tensorflow site.

In the next notebooks I'll cover the others TFX features (Transform, Model Analysis, Serving and API)